<a href="https://colab.research.google.com/github/Vicente-Hernandez/simulador-credito-automotriz/blob/main/simulador_credito_automotriz_fase_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# =============================================================================
# PASO 1: INSTALACIÓN E IMPORTACIÓN DE LIBRERÍAS
# =============================================================================
!pip install pandas openpyxl numpy -q

import pandas as pd
import numpy as np
from datetime import datetime
from dataclasses import dataclass
from enum import Enum
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')
np.random.seed(42)

# =============================================================================
# PASO 2: CLASE DE CONFIGURACIÓN CENTRALIZADA
# =============================================================================
class Config:
    """
    OBJETIVO: Centralizar todas las reglas de negocio, parámetros y datos estáticos.
    PAPEL: Actúa como la "constitución" o el "cerebro" del simulador. Permite modificar
           la lógica de negocio fácilmente sin tocar el código principal.
    """

    # Listas de precios fijas para cada período de tiempo.
    PRECIOS_2024 = {
        'WVR EX 1.5 AUT SMART': {'PRECIO_CONTADO': 18590000, 'PRECIO_FINANCIADO': 17156000}, 'WVR EX 1.5 AUT': {'PRECIO_CONTADO': 18190000, 'PRECIO_FINANCIADO': 16690000},
        'CAMIONETA CUMBRE RTL 3.5': {'PRECIO_CONTADO': 53538100, 'PRECIO_FINANCIADO': 52348100}, 'ZVR TOURING 2.0 4X2': {'PRECIO_CONTADO': 32490000, 'PRECIO_FINANCIADO': 31990000},
        'WVR LX 1.5 AUT': {'PRECIO_CONTADO': 16890000, 'PRECIO_FINANCIADO': 15590000}, 'RVH LX 1.5 AUT.': {'PRECIO_CONTADO': 21990000, 'PRECIO_FINANCIADO': 20990000},
        'CIUDAD HATCHBACK': {'PRECIO_CONTADO': 19490000, 'PRECIO_FINANCIADO': 18990000}, 'VICIV TOURING 1.5': {'PRECIO_CONTADO': 30390000, 'PRECIO_FINANCIADO': 29390000},
        'RVH EX 1.5 AUT': {'PRECIO_CONTADO': 24490000, 'PRECIO_FINANCIADO': 23990000}, 'PLIOT ELITE 3.5': {'PRECIO_CONTADO': 57990000, 'PRECIO_FINANCIADO': 56990000},
        'WVR LX 1.5 MECANICO': {'PRECIO_CONTADO': 13490000, 'PRECIO_FINANCIADO': 12490000}, 'RVH EXL 1.5 AUT': {'PRECIO_CONTADO': 26490000, 'PRECIO_FINANCIADO': 25990000},
        'CIUDAD SEDAN EXL': {'PRECIO_CONTADO': 19490000, 'PRECIO_FINANCIADO': 18990000}, 'VRC EXT 1.5 TURBO': {'PRECIO_CONTADO': 34990000, 'PRECIO_FINANCIADO': 34490000},
        'WVR EXL 1.5 AUT': {'PRECIO_CONTADO': 18990000, 'PRECIO_FINANCIADO': 17490000}, 'VRC TOURING 1.5 4X4': {'PRECIO_CONTADO': 39990000, 'PRECIO_FINANCIADO': 39490000},
        'VICIV EXT 1.5 TURBO': {'PRECIO_CONTADO': 26390000, 'PRECIO_FINANCIADO': 25390000}, 'ZVR EXL 2.0 AUT': {'PRECIO_CONTADO': 30990000, 'PRECIO_FINANCIADO': 30490000},
        'WVR LX 1.5 MECANICO SMART': {'PRECIO_CONTADO': 13990000, 'PRECIO_FINANCIADO': 12990000}, 'WVR EXL 1.5 AUT SMART': {'PRECIO_CONTADO': 39973000, 'PRECIO_FINANCIADO': 42396000},
        'PLIOT TOURING 3.5': {'PRECIO_CONTADO': 55990000, 'PRECIO_FINANCIADO': 54990000}
    }
    PRECIOS_2025 = {
        'WVR EX 1.5 AUT SMART': {'PRECIO_CONTADO': 19290000, 'PRECIO_FINANCIADO': 16790000}, 'WVR EX 1.5 AUT': {'PRECIO_CONTADO': 18490000, 'PRECIO_FINANCIADO': 15990000},
        'ZVR TOURING 2.0 4X2': {'PRECIO_CONTADO': 32490000, 'PRECIO_FINANCIADO': 30990000}, 'WVR LX 1.5 AUT': {'PRECIO_CONTADO': 16990000, 'PRECIO_FINANCIADO': 14990000},
        'RVH LX 1.5 AUT.': {'PRECIO_CONTADO': 23990000, 'PRECIO_FINANCIADO': 21990000}, 'VICIV TOURING 1.5': {'PRECIO_CONTADO': 30390000, 'PRECIO_FINANCIADO': 28390000},
        'RVH EX 1.5 AUT': {'PRECIO_CONTADO': 24990000, 'PRECIO_FINANCIADO': 23990000}, 'PLIOT ELITE 3.5': {'PRECIO_CONTADO': 59990000, 'PRECIO_FINANCIADO': 58990000},
        'RVH EXL 1.5 AUT': {'PRECIO_CONTADO': 26490000, 'PRECIO_FINANCIADO': 25990000}, 'CIUDAD SEDAN EXL': {'PRECIO_CONTADO': 17990000, 'PRECIO_FINANCIADO': 16990000},
        'VRC ADVANCE HYBRID Aut. 4X4': {'PRECIO_CONTADO': 48990000, 'PRECIO_FINANCIADO': 46990000}, 'WVR EXL 1.5 AUT': {'PRECIO_CONTADO': 19490000, 'PRECIO_FINANCIADO': 16990000},
        'VRC TOURING 1.5 4X4': {'PRECIO_CONTADO': 40990000, 'PRECIO_FINANCIADO': 38990000}, 'ZVR EXL 2.0 AUT': {'PRECIO_CONTADO': 30490000, 'PRECIO_FINANCIADO': 28990000},
        'WVR EXL 1.5 AUT SMART': {'PRECIO_CONTADO': 20290000, 'PRECIO_FINANCIADO': 17790000}, 'PLIOT TOURING 3.5': {'PRECIO_CONTADO': 57990000, 'PRECIO_FINANCIADO': 56990000}
    }

    # Catálogo de modelos y políticas de disponibilidad.
    MAPEO_MODELOS = {
        'WVR': ['WVR LX 1.5 MECANICO', 'WVR LX 1.5 MECANICO SMART', 'WVR LX 1.5 AUT', 'WVR EX 1.5 AUT', 'WVR EXL 1.5 AUT', 'WVR EX 1.5 AUT SMART', 'WVR EXL 1.5 AUT SMART'],
        'RVH': ['RVH LX 1.5 AUT.', 'RVH EX 1.5 AUT', 'RVH EXL 1.5 AUT'], 'VICIV': ['VICIV EXT 1.5 TURBO', 'VICIV TOURING 1.5'],
        'CIUDAD': ['CIUDAD SEDAN EXL', 'CIUDAD HATCHBACK'], 'VRC': ['VRC EXT 1.5 TURBO', 'VRC TOURING 1.5 4X4', 'VRC ADVANCE HYBRID Aut. 4X4'],
        'CAMIONETA': ['CAMIONETA CUMBRE RTL 3.5'], 'PLIOT': ['PLIOT TOURING 3.5', 'PLIOT ELITE 3.5'],
        'ZVR': ['ZVR EXL 2.0 AUT', 'ZVR TOURING 2.0 4X2']
    }
    MODELOS_DISCONTINUADOS_2025 = ['CAMIONETA CUMBRE RTL 3.5', 'WVR LX 1.5 MECANICO', 'WVR LX 1.5 MECANICO SMART', 'CIUDAD HATCHBACK', 'VICIV EXT 1.5 TURBO', 'VRC EXT 1.5 TURBO']

    # Estructura de tasas de interés por período y tipo de vehículo.
    ESTRUCTURA_TASAS = {
        'periodo_1': {'fecha_fin': datetime(2025, 3, 31), 'tasas_mercado': {0.0256: 0.15, 0.0199: 0.65, 0.0179: 0.20}, 'tasa_gama_alta': 0.0159},
        'periodo_2': {'fecha_fin': datetime(2026, 12, 31), 'tasas_mercado': {0.0256: 0.15, 0.0179: 0.65, 0.0159: 0.20}, 'tasa_gama_alta': 0.0139}
    }

    # Preferencias de los clientes en cuanto a plazos de financiamiento.
    PLAZOS_PREFERENCIA = {
        36: 0.45,  # La preferencia más alta, que permite una buen balance entre prestación/retoma (renovación)
        48: 0.30,
        24: 0.20,
        60: 0.05   # Último recurso, cuando el cliente debe hacer calzar sus ingresos con valor de cuota.
    }

    # Perfiles de cliente (arquetipos) con sus características y probabilidad de aparición.
    ARQUETIPOS_CLIENTE = {
        'Patrimonio Consolidado': {'probabilidad': 0.15, 'params_renta': {'mean': 4500000, 'std': 800000}, 'params_score': {'mean': 800, 'std': 40}, 'prob_cliente_antiguo': 0.75},
        'Profesional Endeudado': {'probabilidad': 0.20, 'params_renta': {'mean': 3200000, 'std': 600000}, 'params_score': {'mean': 670, 'std': 50}, 'prob_cliente_antiguo': 0.35},
        'Clase Media Estable': {'probabilidad': 0.40, 'params_renta': {'mean': 2200000, 'std': 400000}, 'params_score': {'mean': 750, 'std': 60}, 'prob_cliente_antiguo': 0.25},
        'Clase Media Emergente': {'probabilidad': 0.25, 'params_renta': {'mean': 1600000, 'std': 200000}, 'params_score': {'mean': 620, 'std': 70}, 'prob_cliente_antiguo': 0.05}
    }

    # Red de concesionarios y sucursales.
    ESTRUCTURA_COMERCIAL = {
        'AK - Bilbao': {'comuna': 'La Reina', 'vendedores': ['Rodrigo Vidal', 'Andres Tobar']}, 'SAI - La Dehesa': {'comuna': 'Lo Barnechea', 'vendedores': ['Ricardo Valenzuela', 'Camila Torres']},
        'SAI - Las Condes': {'comuna': 'Las Condes', 'vendedores': ['Marcelo Carvallo', 'Anais Urdaneta']}, 'SAI - Movicenter': {'comuna': 'Huechuraba', 'vendedores': ['Adolfo Bravo', 'Debora Avila', 'Jesus Castillo']},
        'SAI - La Florida': {'comuna': 'La Florida', 'vendedores': ['Monica Fuentes', 'Felipe Gutierrez']}, 'SAI - Costanera': {'comuna': 'Providencia', 'vendedores': ['Daniel Lopez', 'Carolina Saavedra']},
        'SAI - Plaza Oeste': {'comuna': 'Cerrillos', 'vendedores': ['Esteban Paillao', 'Patricia Ramirez']}, 'V&D - Padre Hurtado': {'comuna': 'Las Condes', 'vendedores': ['Alejandra Paillamilla', 'Tatiana Jarusauskas']},
        'V&D - Manuel Montt': {'comuna': 'Providencia', 'vendedores': ['Angel Ramos', 'Daniela Morales']}, 'V&D - Irarrazaval': {'comuna': 'Nunoa', 'vendedores': ['Cesar Hernandez', 'Antonio Bustamante']}
    }
    PROBABILIDADES_VOLUMEN_SUCURSAL = {'SAI - Plaza Oeste': 0.16, 'SAI - Movicenter': 0.16, 'V&D - Manuel Montt': 0.16, 'AK - Bilbao': 0.11, 'V&D - Padre Hurtado': 0.11, 'SAI - La Dehesa': 0.06, 'SAI - Las Condes': 0.06, 'SAI - La Florida': 0.06, 'SAI - Costanera': 0.06, 'V&D - Irarrazaval': 0.06}
    PERFILES_SUCURSAL = {'alto_valor': ['SAI - Movicenter', 'V&D - Manuel Montt', 'AK - Bilbao', 'V&D - Padre Hurtado'], 'alto_volumen': ['SAI - Plaza Oeste']}

    # Información adicional de negocios: definición de alta gama y preferencias de modelos.
    MODELOS_ALTA_GAMA = ['PLIOT TOURING 3.5', 'PLIOT ELITE 3.5', 'VRC EXT 1.5 TURBO', 'VRC TOURING 1.5 4X4', 'VRC ADVANCE HYBRID Aut. 4X4']
    MODELOS_DISTRIBUCION = {'WVR': 0.28, 'RVH': 0.22, 'VICIV': 0.18, 'CIUDAD': 0.15, 'VRC': 0.10, 'CAMIONETA': 0.04, 'PLIOT': 0.02, 'ZVR': 0.01}

    # Contexto macroeconómico simulado.
    CONTEXTO_SIMULADO = {'2024-01':{'desempleo_tasa': 0.085}, '2024-02':{'desempleo_tasa': 0.088}, '2024-03':{'desempleo_tasa': 0.086}, '2024-04':{'desempleo_tasa': 0.084}, '2024-05':{'desempleo_tasa': 0.082}, '2024-06':{'desempleo_tasa': 0.081}, '2024-07':{'desempleo_tasa': 0.079}, '2024-08':{'desempleo_tasa': 0.077}, '2024-09':{'desempleo_tasa': 0.076}, '2024-10':{'desempleo_tasa': 0.074}, '2024-11':{'desempleo_tasa': 0.072}, '2024-12':{'desempleo_tasa': 0.070}, '2025-01':{'desempleo_tasa': 0.068}, '2025-02':{'desempleo_tasa': 0.066}, '2025-03':{'desempleo_tasa': 0.064}, '2025-04':{'desempleo_tasa': 0.062}, '2025-05':{'desempleo_tasa': 0.060}, '2025-06':{'desempleo_tasa': 0.058}, '2025-07':{'desempleo_tasa': 0.056}}

    # Define la estructura de datos para un cliente.
    @dataclass
    class PerfilCliente:
        edad: int; renta: float; score_crediticio: int; antiguedad_laboral: int;
        cliente_antiguo_marca: bool; arquetipo: str

# =============================================================================
# PASO 3: CLASE GENERADORA DEL MOTOR DE SIMULACIÓN
# =============================================================================
class GeneradorMotorCo:
    """
    OBJETIVO: Ser la "fábrica" que construye los registros de operaciones de crédito.
    PAPEL: Contiene toda la lógica procedural para generar un registro, utilizando
           las reglas definidas en la clase `Config`.
    """
    def __init__(self, start_date: datetime = datetime(2024, 1, 1), end_date: datetime = datetime(2025, 7, 31)):
        """
        OBJETIVO: Inicializar el generador.
        PAPEL: Define el marco de tiempo de la simulación y la estacionalidad de las ventas.
        """
        self.start_date = start_date; self.end_date = end_date
        self.distribucion_mensual = {1:0.08, 2:0.06, 3:0.07, 4:0.05, 5:0.05, 6:0.05, 7:0.07, 8:0.12, 9:0.08, 10:0.1, 11:0.1, 12:0.17}

    def _seleccionar_sucursal(self) -> str:
        """
        OBJETIVO: Elegir una sucursal para la venta.
        CÓMO: Realiza una selección aleatoria ponderada según las probabilidades de volumen de `Config`.
        """
        sucursales = list(Config.PROBABILIDADES_VOLUMEN_SUCURSAL.keys())
        probabilidades = list(Config.PROBABILIDADES_VOLUMEN_SUCURSAL.values())
        return np.random.choice(sucursales, p=probabilidades)

    def _seleccionar_categoria_modelo_influenciada(self, sucursal: str):
        """
        OBJETIVO: Elegir una categoría de vehículo (ej. 'SUV', 'Sedan').
        CÓMO: Ajusta las probabilidades base de venta de cada categoría según el perfil de la
               sucursal ('alto_valor' vs 'alto_volumen') antes de la selección aleatoria.
        """
        prob_base = Config.MODELOS_DISTRIBUCION.copy()
        if sucursal in Config.PERFILES_SUCURSAL['alto_valor']:
            for modelo in ['VRC', 'PLIOT', 'ZVR', 'CAMIONETA']: prob_base[modelo] *= 2.0
            for modelo in ['CIUDAD']: prob_base[modelo] *= 0.5
        elif sucursal in Config.PERFILES_SUCURSAL['alto_volumen']:
            for modelo in ['CIUDAD', 'WVR']: prob_base[modelo] *= 1.8
            for modelo in ['VRC', 'PLIOT', 'ZVR']: prob_base[modelo] *= 0.3
        total = sum(prob_base.values())
        prob_ajustada = {k: v / total for k, v in prob_base.items()}
        return np.random.choice(list(prob_ajustada.keys()), p=list(prob_ajustada.values()))

    def _seleccionar_tasa_de_mercado(self, fecha: datetime, modelo: str):
        """
        OBJETIVO: Asignar una tasa de interés.
        CÓMO: Selecciona la estructura de tasas correcta (2024 o 2025) según la fecha. Si es un
               auto de gama alta, aplica la tasa fija de campaña. Si no, elige una tasa de
               mercado según las probabilidades definidas en `Config`.
        """
        periodo_actual = Config.ESTRUCTURA_TASAS['periodo_1'] if fecha <= Config.ESTRUCTURA_TASAS['periodo_1']['fecha_fin'] else Config.ESTRUCTURA_TASAS['periodo_2']
        es_gama_alta = any(gama in modelo for gama in Config.MODELOS_ALTA_GAMA)
        if es_gama_alta: return periodo_actual['tasa_gama_alta']
        tasas = list(periodo_actual['tasas_mercado'].keys())
        probabilidades = list(periodo_actual['tasas_mercado'].values())
        return np.random.choice(tasas, p=probabilidades)

    def _seleccionar_plazo_viable(self, monto_capital, tasa_mensual, cuoton_monto, renta_cliente):
        """
        OBJETIVO: Encontrar un plazo viable que respete las preferencias del mercado.
        CÓMO: Primero, identifica todos los plazos (24, 36, 48, 60) que el cliente puede pagar
               sin superar el 33% de su renta. Luego, elige uno de esos plazos viables
               basándose en las probabilidades de preferencia de `Config`.
        """
        cuota_maxima = renta_cliente * 0.33
        plazos_viables = []

        for plazo in Config.PLAZOS_PREFERENCIA.keys():
            cuota_calculada = self._calcular_cuota_mensual(monto_capital, tasa_mensual * 12, plazo, cuoton_monto)
            if cuota_calculada <= cuota_maxima:
                plazos_viables.append(plazo)

        if not plazos_viables:
            return None

        preferencias_filtradas = {p: Config.PLAZOS_PREFERENCIA[p] for p in plazos_viables}
        total_prob = sum(preferencias_filtradas.values())
        prob_normalizadas = [v / total_prob for v in preferencias_filtradas.values()]

        return np.random.choice(list(preferencias_filtradas.keys()), p=prob_normalizadas)

    def _generar_perfil_cliente(self, fecha_operacion: datetime):
        """
        OBJETIVO: Crear un perfil de cliente realista y complejo.
        CÓMO: Selecciona un "arquetipo", genera la renta y el score a partir de las
               estadísticas de ese arquetipo, y aplica reglas de negocio como el sueldo mínimo.
        """
        arquetipos = list(Config.ARQUETIPOS_CLIENTE.keys())
        probabilidades = [v['probabilidad'] for v in Config.ARQUETIPOS_CLIENTE.values()]
        arquetipo_elegido = np.random.choice(arquetipos, p=probabilidades)
        params = Config.ARQUETIPOS_CLIENTE[arquetipo_elegido]
        renta_generada = np.random.normal(loc=params['params_renta']['mean'], scale=params['params_renta']['std'])
        score_generado = int(np.random.normal(loc=params['params_score']['mean'], scale=params['params_score']['std']))
        sueldo_minimo = 529000 if fecha_operacion >= datetime(2025, 5, 1) else 500000
        renta_final = max(sueldo_minimo, renta_generada)
        renta_final_formateada = round(renta_final / 500) * 500
        es_cliente_antiguo = np.random.random() < params['prob_cliente_antiguo']
        return Config.PerfilCliente(
            edad=int(np.random.uniform(25, 65)), renta=renta_final_formateada,
            score_crediticio=np.clip(score_generado, 450, 900), antiguedad_laboral=int(np.random.uniform(1, 20)),
            cliente_antiguo_marca=es_cliente_antiguo, arquetipo=arquetipo_elegido
        )

    def _calcular_probabilidad_default(self, perfil, cuota, contexto):
        """
        OBJETIVO: Calcular la probabilidad de que un crédito termine en impago (PD).
        CÓMO: Utiliza un modelo logístico que pondera el score, la carga financiera, el
               desempleo y si el cliente es antiguo en la marca.
        """
        score_norm = (perfil.score_crediticio - 700) / 100
        ratio_cuota_renta = min(cuota / perfil.renta, 1) if perfil.renta > 0 else 1
        ajuste_cliente_antiguo = -0.5 if perfil.cliente_antiguo_marca else 0.0
        desempleo_mes = contexto.get('desempleo_tasa', 0.07)
        z = -1.5 - 1.5 * score_norm + 2.5 * ratio_cuota_renta + 0.8 * (desempleo_mes - 0.07) + ajuste_cliente_antiguo
        return 1 / (1 + np.exp(-z))

    def _seleccionar_modelo_especifico(self, categoria: str, fecha: datetime):
        """
        OBJETIVO: Seleccionar un modelo de vehículo específico.
        CÓMO: Filtra el catálogo, quitando modelos descontinuados o aún no lanzados según la fecha.
        """
        modelos_disponibles = Config.MAPEO_MODELOS.get(categoria, [])
        if fecha.year >= 2025:
            modelos_disponibles = [m for m in modelos_disponibles if m not in Config.MODELOS_DISCONTINUADOS_2025]
        else:
            modelos_disponibles = [m for m in modelos_disponibles if 'HYBRID' not in m]
        return np.random.choice(modelos_disponibles) if modelos_disponibles else None

    def generar_registro(self):
        """
        OBJETIVO: Orquestar la creación de un registro completo, desde el cliente hasta el crédito.
        PAPEL: Es el corazón de la "fábrica" que ensambla el resultado final, manejando
               toda la lógica de aprobación y rechazo en secuencia.
        """
        fecha_operacion = self._generar_fecha_operacion()
        contexto_macro = Config.CONTEXTO_SIMULADO.get(fecha_operacion.strftime('%Y-%m'), {})

        perfil = self._generar_perfil_cliente(fecha_operacion)

        base_record = {
            'FECHA_OPERACION': fecha_operacion, 'CONCESIONARIO': 'N/A', 'SUCURSAL': 'N/A',
            'VENDEDOR': 'N/A', 'MODELO': 'N/A', 'TIPO_COMPRA': 'N/A',
            'CLIENTE_ARQUETIPO': perfil.arquetipo, 'CLIENTE_RENTA': int(perfil.renta),
            'CLIENTE_SCORE': perfil.score_crediticio, 'CLIENTE_ANTIGUO_MARCA': perfil.cliente_antiguo_marca,
            'ESTADO': 'Rechazado', 'MOTIVO_RECHAZO': 'N/A', 'PRECIO_FINAL': 0,
            'TIPO_FINANCIAMIENTO': 'N/A', 'MONTO_CAPITAL': 0, 'PLAZO_MESES': 0, 'TASA_MENSUAL': 0,
            'PIE_PORCENTAJE': 0, 'PIE_MONTO': 0, 'CUOTON_PORCENTAJE': 0, 'CUOTA_MENSUAL': 0,
            'FUTURO_DEFAULT': False
        }

        # --- EVALUACIÓN INICIAL DE RIESGO ---
        if np.random.random() < 0.05:
            base_record['MOTIVO_RECHAZO'] = 'Causas Judiciales'
            return base_record
        if perfil.score_crediticio < 580:
            base_record['MOTIVO_RECHAZO'] = 'Mal Comportamiento Financiero (DICOM)'
            return base_record
        if perfil.antiguedad_laboral < 1:
            base_record['MOTIVO_RECHAZO'] = 'Antiguedad laboral insuficiente'
            return base_record

        # --- LÓGICA DE VENTA ---
        sucursal_venta_str = self._seleccionar_sucursal()
        base_record.update({'CONCESIONARIO': sucursal_venta_str.split(' - ')[0], 'SUCURSAL': sucursal_venta_str.split(' - ')[1]})

        categoria_modelo = self._seleccionar_categoria_modelo_influenciada(sucursal_venta_str)
        modelo_completo = self._seleccionar_modelo_especifico(categoria_modelo, fecha_operacion)
        if not modelo_completo: return None
        base_record['MODELO'] = modelo_completo

        vendedor = np.random.choice(Config.ESTRUCTURA_COMERCIAL[sucursal_venta_str]['vendedores'])
        base_record['VENDEDOR'] = vendedor

        # --- LÓGICA DE PRECIOS Y TIPO DE COMPRA ---
        lista_precios_del_ano = Config.PRECIOS_2024 if fecha_operacion.year < 2025 else Config.PRECIOS_2025
        precios_modelo = lista_precios_del_ano.get(modelo_completo)
        if not precios_modelo: return None

        tipo_compra = 'Contado' if np.random.random() < 0.32 else 'Financiado'
        base_record['TIPO_COMPRA'] = tipo_compra

        if tipo_compra == 'Contado':
            base_record.update({ 'ESTADO': 'Aprobado', 'PRECIO_FINAL': precios_modelo['PRECIO_CONTADO'], 'PIE_PORCENTAJE': 100, 'PIE_MONTO': precios_modelo['PRECIO_CONTADO'] })
            return base_record

        # --- LÓGICA DE FINANCIAMIENTO Y APROBACIÓN FINAL ---
        precio_financiado = precios_modelo['PRECIO_FINANCIADO']
        base_record['PRECIO_FINAL'] = precio_financiado
        opciones_pie = [20, 30, 40] if fecha_operacion < datetime(2025, 4, 1) else [20, 30, 40, 50]
        pie_pct = np.random.choice(opciones_pie)
        pie_monto = round(precio_financiado * (pie_pct / 100), -3)
        monto_capital = round(precio_financiado - pie_monto, -3)

        tipo_financiamiento = 'Renovacion' if np.random.random() < 0.7 else 'Tradicional'
        cuoton_pct = np.random.choice([30, 40, 50]) if tipo_financiamiento == 'Renovacion' else 0
        cuoton_monto = monto_capital * (cuoton_pct / 100)

        tasa_interes_mensual = self._seleccionar_tasa_de_mercado(fecha_operacion, modelo_completo)

        plazo_final = self._seleccionar_plazo_viable(monto_capital, tasa_interes_mensual, cuoton_monto, perfil.renta)

        if plazo_final is None:
            base_record['MOTIVO_RECHAZO'] = 'Carga financiera excede el 33%'
            return base_record

        cuota_mensual = self._calcular_cuota_mensual(monto_capital, tasa_interes_mensual * 12, plazo_final, cuoton_monto)
        prob_default = self._calcular_probabilidad_default(perfil, cuota_mensual, contexto_macro)
        futuro_default = np.random.random() < prob_default

        base_record.update({
            'ESTADO': 'Aprobado', 'MOTIVO_RECHAZO': 'N/A', 'TIPO_FINANCIAMIENTO': tipo_financiamiento,
            'MONTO_CAPITAL': int(monto_capital), 'PLAZO_MESES': plazo_final, 'TASA_MENSUAL': round(tasa_interes_mensual, 5),
            'PIE_PORCENTAJE': pie_pct, 'PIE_MONTO': int(pie_monto), 'CUOTON_PORCENTAJE': cuoton_pct,
            'CUOTA_MENSUAL': int(cuota_mensual), 'FUTURO_DEFAULT': futuro_default
        })
        return base_record

    def _generar_fecha_operacion(self):
        """Genera una fecha aleatoria para la operación, con ponderación estacional."""
        while True:
            mes = np.random.choice(list(self.distribucion_mensual.keys()), p=list(self.distribucion_mensual.values()))
            año = np.random.choice([2024, 2025], p=[0.6, 0.4])
            dia = np.random.randint(1, 29)
            fecha = datetime(año, mes, dia)
            if self.start_date <= fecha <= self.end_date: return fecha

    def _calcular_cuota_mensual(self, monto, tasa_anual, plazo, cuoton=0):
        """Calcula la cuota mensual usando la fórmula de amortización francesa."""
        if monto <= 0 or plazo <= 0: return 0
        tasa_mensual = tasa_anual / 12
        if tasa_mensual == 0: return (monto - cuoton) / plazo
        if cuoton > 0:
            return round((monto - cuoton / ((1 + tasa_mensual) ** plazo)) * (tasa_mensual * (1 + tasa_mensual) ** plazo) / (((1 + tasa_mensual) ** plazo) - 1))
        else:
            return round(monto * (tasa_mensual * (1 + tasa_mensual) ** plazo) / (((1 + tasa_mensual) ** plazo) - 1))

# =============================================================================
# PASO 4: FUNCIONES DE EJECUCIÓN
# =============================================================================
def generar_dataset_por_kpi(min_registros_validos=5000, kpi_objetivo_min=0.60, kpi_objetivo_max=0.65, lote=1000):
    """
    OBJETIVO: Dirigir la simulación para que cumpla un objetivo de negocio.
    PAPEL: Ejecuta el generador en un bucle hasta que la cartera de créditos aprobados
           alcance el KPI de proporción de financiamiento (60-65%).
    """
    print(f"Iniciando generación dinámica hasta alcanzar KPI de financiamiento ({kpi_objetivo_min*100}%-{kpi_objetivo_max*100}%)...")
    generador = GeneradorMotorCo()
    registros_totales = []

    pbar = tqdm(desc="Generando Lotes", unit="lote")
    while True:
        nuevos_registros = [generador.generar_registro() for _ in range(lote)]
        registros_totales.extend(r for r in nuevos_registros if r is not None)

        df_temp = pd.DataFrame(registros_totales)
        df_aprobados = df_temp[df_temp['ESTADO'] == 'Aprobado']

        if not df_aprobados.empty:
            conteo_tipos = df_aprobados['TIPO_COMPRA'].value_counts(normalize=True)
            prop_financiado = conteo_tipos.get('Financiado', 0)
        else: prop_financiado = 0

        num_validos = len(df_aprobados)
        pbar.update(1)
        pbar.set_postfix_str(f"{num_validos} válidos, {prop_financiado:.2%} financiado")

        if num_validos >= min_registros_validos and kpi_objetivo_min <= prop_financiado <= kpi_objetivo_max:
            break

    pbar.close()
    df_final = pd.DataFrame(registros_totales)
    print(f"\n KPI alcanzado. Generación completa.")
    return df_final

def analizar_y_exportar(df: pd.DataFrame, nombre_archivo: str = 'dataset.xlsx'):
    """
    OBJETIVO: Presentar un resumen de los resultados y guardar el dataset.
    CÓMO: Muestra análisis de aprobación y rechazo, y guarda el dataset en un
           archivo Excel con dos pestañas (completo y solo aprobados).
    """
    print("\n Realizando análisis")
    try:
        df_aprobados = df[df['ESTADO'] == 'Aprobado']

        print("\n--- Proporción por Tipo de Compra (Aprobados) ---")
        print(df_aprobados['TIPO_COMPRA'].value_counts(normalize=True).round(2))

        print("\n--- Análisis de Rechazos ---")
        if 'Rechazado' in df['ESTADO'].unique():
            print(df[df['ESTADO'] == 'Rechazado']['MOTIVO_RECHAZO'].value_counts())
        else:
            print("No se registraron operaciones rechazadas.")

        print("\n Exportando dataset completo (Aprobados y Rechazados) a Excel.")
        with pd.ExcelWriter(nombre_archivo) as writer:
            df.to_excel(writer, sheet_name='Dataset_Completo', index=False)
            df_aprobados.to_excel(writer, sheet_name='Solo_Aprobados', index=False)
        print(f"El archivo '{nombre_archivo}' ha sido guardado con dos pestañas.")

    except Exception as e:
        print(f"Error Crítico durante el análisis o exportación: {e}")

# =============================================================================
# PASO 5: ¡EJECUTAR LA SIMULACIÓN!
# =============================================================================
if __name__ == '__main__':
    """
    OBJETIVO: Iniciar todo el proceso de simulación.
    PAPEL: Es el punto de entrada principal del script.
    """
    try:
        dataset_final = generar_dataset_por_kpi()
        if not dataset_final.empty:
            analizar_y_exportar(dataset_final, nombre_archivo='dataset.xlsx')
    except Exception as e:
        print(f"ERROR FATAL: El proceso de simulación no pudo completarse.")
        print(f"   Detalle: {e}")

Iniciando generación dinámica hasta alcanzar KPI de financiamiento (60.0%-65.0%)...


Generando Lotes: 7lote [00:03,  2.10lote/s, 5469 válidos, 64.34% financiado]



 KPI alcanzado. Generación completa.

 Realizando análisis

--- Proporción por Tipo de Compra (Aprobados) ---
TIPO_COMPRA
Financiado    0.64
Contado       0.36
Name: proportion, dtype: float64

--- Análisis de Rechazos ---
MOTIVO_RECHAZO
Carga financiera excede el 33%           570
Mal Comportamiento Financiero (DICOM)    533
Causas Judiciales                        363
Name: count, dtype: int64

 Exportando dataset completo (Aprobados y Rechazados) a Excel.
El archivo 'dataset.xlsx' ha sido guardado con dos pestañas.
